# DSC350 - Week 10 - Exercise 10.2

Complete the following exercises and submit them to the assignment link. For all exercises in this course, you can either submit a Jupyter Notebook for a PDF of your code. If you submit a .py file you need to also include a PDF or attachment of your results.

**1. Import sqlite3 and create a cursor**
 - a) Create the sensors table (pg 180)
 - b) Write a query to confirm your table was created - print your results
 - c) Insert random data into the table you created and then select all the results from the table (Select*)
 - d) Drop the table and confirm it has been dropped based on your previous query
 - e) Close your connection

In [1]:
# Import the necessary library
import sqlite3

# Connect to SQLite DB 
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

# Create sensor table
cursor.execute('''CREATE TABLE sensors (date text, city text, code text, sensor_id real, temperature real)''')

# Confirm table was created and print results
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='sensors';")
table_created = cursor.fetchall()
print("Table created:", table_created)

# Inset data to table
cursor.execute('''INSERT INTO sensors (date, city, code, sensor_id, temperature) VALUES (?, ?, ?, ?, ?)''', ('2016-11-05', 'Utrecht', 'Red', 42, 15.14))

# Commit changes
conn.commit()

# Drop the sensors table
cursor.execute('DROP TABLE sensors')

# Confirm table was dropped
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='sensors';")
table_dropped = cursor.fetchall()
print("Table dropped:", table_dropped)

# Close connection
conn.close()

Table created: [('sensors',)]
Table dropped: []


**2. Create a list of tuples to form the pandas DataFrame**
 - a) Create a table without specifying data types
 - b) Insert records from a list of tuples - insert all the rows into the table and show the row count
 - c) Delete the records where the sun activity is greater than 20
 - d) Write a query to select all data where the year is less than 1732

In [2]:
# Import necessary libraries
import statsmodels.api as sm
import pandas as pd
from pandas.io.sql import read_sql
import sqlite3

# Load data to pandas DataFrame
data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
rows = [tuple(x) for x in df.values]

# Connect SQLite
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create table
cursor.execute('CREATE TABLE sunspots (year, sunactivity)')

# Insert records from DataFrame into table
df.to_sql('sunspots', conn, if_exists='append', index=False)

# Show row count
cursor.execute('SELECT COUNT(*) FROM sunspots')
row_count = cursor.fetchone()[0]
print(f'Row count before deletion: {row_count}')

# Delete records based on specific criteria
cursor.execute('DELETE FROM sunspots WHERE sunactivity > 20')

# Show row count after deletion
cursor.execute('SELECT COUNT(*) FROM sunspots')
row_count = cursor.fetchone()[0]
print(f'Row count after deletion: {row_count}')

# Select all data where year < 1732
query = 'SELECT * FROM sunspots WHERE year < 1732'
result_df = pd.read_sql(query, conn)

# Display results and close connection
print(result_df)
conn.close()

Row count before deletion: 309
Row count after deletion: 92
      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1707.0         20.0
4   1708.0         10.0
5   1709.0          8.0
6   1710.0          3.0
7   1711.0          0.0
8   1712.0          0.0
9   1713.0          2.0
10  1714.0         11.0
11  1723.0         11.0


**3. Install sqlalchemy**
 - a) Define the superclass
 - b) Create the DBSession
 - c) Create 2 stations and commit the session
 - d) Insert a sensor record

In [3]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import warnings
warnings.filterwarnings('ignore')

# Define the superclass
Base = declarative_base()

# Define the Station class
class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    location = Column(String, nullable=False)

# Define the Sensor class
class Sensor(Base):
    __tablename__ = 'sensors'
    id = Column(Integer, primary_key=True, autoincrement=True)
    type = Column(String, nullable=False)
    station_id = Column(Integer, ForeignKey('stations.id'), nullable=False)

    # Define relationship between Station and Sensor
    station = relationship('Station', back_populates='sensors')

# Add reverse relationship in Station class
Station.sensors = relationship('Sensor', order_by=Sensor.id, back_populates='station')

# Define function to popluate with engine
def populate(engine):
    # Create all tables in database
    Base.metadata.bind(engine)
    # Create configured DBSession class
    DBSession = sessionmaker(bind=engine)
    # Create a Session
    session = DBSession()

    # Create two stations
    de_bilt = Station(name='De Bilt', location='Netherlands')
    utrecht = Station(name='Utrecht', location='Netherlands')
    # Add stations to session
    session.add(de_bilt)
    session.add(utrecht)
    # Commit to database
    session.commit()

    # Insert a sensor record
    temp_sensor = Sensor(type='Temperature', station_id=de_bilt.id)
    # Add sensor to session
    session.add(temp_sensor)
    # Commit to database
    session.commit()

if __name__ == "__main__":
    print("This script is used by code further down in this notebook")

This script is used by code further down in this notebook


**4. Create an engine from a URI**
 - a) Create the station and sensor tables with the engine we just created
 - b) Create a DBSession object
 - c) Select the first row in the station table
 - d) Select all the stations
 - e) Select all the sensors
 - f) Select the first sensor, which belongs to the first station
 - g) Query the station table with the pandas read_sql() 

In [4]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import pandas as pd
import os


# Define the superclass
Base = declarative_base()

# Define the Station class
class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    location = Column(String, nullable=False)

# Define the Sensor class
class Sensor(Base):
    __tablename__ = 'sensors'
    id = Column(Integer, primary_key=True, autoincrement=True)
    type = Column(String, nullable=False)
    station_id = Column(Integer, ForeignKey('stations.id'), nullable=False)

    # Define relationship between Station and Sensor
    station = relationship('Station', back_populates='sensors')

# Add reverse relationship in Station class
Station.sensors = relationship('Sensor', order_by=Sensor.id, back_populates='station')

# Create SQLite DB in memory
engine = create_engine('sqlite:///demo.db')

# Create all tables in database
Base.metadata.create_all(engine)

# Create configured DBSession class
DBSession = sessionmaker(bind=engine)

# Create a Session
session = DBSession()

# Create two stations
de_bilt = Station(name='De Bilt', location='Netherlands')
utrecht = Station(name='Utrecht', location='Netherlands')
# Add stations to session
session.add(de_bilt)
session.add(utrecht)
# Commit to database
session.commit()

# Insert a sensor record
temp_sensor = Sensor(type='Temperature', station_id=de_bilt.id)
# Add sensor to session
session.add(temp_sensor)
# Commit to database
session.commit()

# Select first row in station table
first_station = session.query(Station).first()
print("Query 1 - First station:", first_station.name, first_station.location)

# Select all stations
all_stations = session.query(Station).all()
print("Query 2 - All stations:")
for station in all_stations:
    print(station.name, station.location)

# Select all sensors
all_sensors = session.query(Sensor).all()
print("All sensors:")
for sensor in all_sensors:
    print(sensor.type, sensor.station_id)

# Select the first sensor belonging to first station
first_sensor = session.query(Sensor).filter(Sensor.station_id == first_station.id).first()
print("Query 3 - First sensor of the first station:", first_sensor.type)

# Query station table with pandas read_sql()
df_stations = pd.read_sql('stations', engine)
print("Stations DataFrame:")
print(df_stations)

Base.metadata.drop_all(engine)

# Close session
session.close()

Query 1 - First station: De Bilt Netherlands
Query 2 - All stations:
De Bilt Netherlands
Utrecht Netherlands
All sensors:
Temperature 1
Query 3 - First sensor of the first station: Temperature
Stations DataFrame:
   id     name     location
0   1  De Bilt  Netherlands
1   2  Utrecht  Netherlands


**5. Draft a 500-word paper about the ethical impact transforming data could have on society, within your organization, within your own family, etc.**

Essay was completed and turned in via another medium.